In [1]:
import pandas as pd
from pymongo import MongoClient
import pprint
import re
import numpy as np
import folium

In [246]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [264]:
companies = db.data_companies_clean.find({ 
    "total_money_raised":{
        "$gte": 1000000
    }
}
)  

In [265]:
data_companies = pd.DataFrame(companies)
data_companies

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised
0,5d2737547bf280155f216096,finance,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.1418973, ...",51.510880,-0.141897,Seedcamp,4.0,5000000
1,5d2737547bf280155f216097,tech,Tel-Aviv,ISR,2007,"{'type': 'Point', 'coordinates': [34.7595, 32....",32.055400,34.759500,Flixwagon,10.0,2500000
2,5d2737547bf280155f216098,tech,Toronto,CAN,2007,"{'type': 'Point', 'coordinates': [-79.388279, ...",43.666953,-79.388279,ModiFace,35.0,4250000
3,5d2737547bf280155f216099,tech,Helsinki,FIN,2007,"{'type': 'Point', 'coordinates': [24.9410012, ...",60.168149,24.941001,XIHA,8.0,1000000
4,5d2737547bf280155f21609a,tech,Amsterdam,NLD,2007,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Wakoopa,8.0,1000000
5,5d2737547bf280155f21609c,education,Berlin,DEU,2007,"{'type': 'Point', 'coordinates': [13.38861, 52...",52.489700,13.388610,babbel,205.0,10000000
6,5d2737547bf280155f21609d,tech,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.0934254, ...",51.514157,-0.093425,Zemanta,30.0,7350000
7,5d2737547bf280155f21609e,tech,Madrid,ESP,2007,"{'type': 'Point', 'coordinates': [-3.6952896, ...",40.430940,-3.695290,eRepublik,30.0,2750000
8,5d2737547bf280155f2160a4,tech,Vancouver,CAN,2007,"{'type': 'Point', 'coordinates': [-123.1078045...",49.282108,-123.107805,Strutta,12.0,1600000
9,5d2737547bf280155f2160a5,news,Paris,FRA,2008,"{'type': 'Point', 'coordinates': [2.350987, 48...",48.856667,2.350987,fotopedia,17.0,12700000


In [5]:
data_companies[data_companies['total_money_raised']==data_companies['total_money_raised'].max()]

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised
68,5d2737547bf280155f216109,social,Vancouver,CAN,2008,"{'type': 'Point', 'coordinates': [-123.0952381...",49.285173,-123.095238,HootSuite,370.0,190000000


In [266]:
list_geo = []
for e in range(len(data_companies)):
    list_geo.append(data_companies.geoDescription[e])

In [267]:
def findNear(list_geo, radio_meters):
        geopoint = list_geo
        return list(db.data_companies_clean.find({
        "geoDescription": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }
       }
    }
    )
    ) 

radio_meters = 5000
list_number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  findNear(list_geo[i], radio_meters)
    list_offices.append(num_offices)
    #print(f"Hay {len(num_offices)} oficinas cerca a {radio_max_meters} metros")
    list_number_offices.append(len(num_offices))
    #print(list_number_offices)
    #print("La más cercana es")
    #print(num_offices)
data_companies['number of offices near'] = list_number_offices

In [252]:
display(data_companies)

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised,number of offices near
0,5d2737547bf280155f216096,finance,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.1418973, ...",51.510880,-0.141897,Seedcamp,4.0,5000000,18
1,5d2737547bf280155f216097,tech,Tel-Aviv,ISR,2007,"{'type': 'Point', 'coordinates': [34.7595, 32....",32.055400,34.759500,Flixwagon,10.0,2500000,5
2,5d2737547bf280155f216098,tech,Toronto,CAN,2007,"{'type': 'Point', 'coordinates': [-79.388279, ...",43.666953,-79.388279,ModiFace,35.0,4250000,5
3,5d2737547bf280155f216099,tech,Helsinki,FIN,2007,"{'type': 'Point', 'coordinates': [24.9410012, ...",60.168149,24.941001,XIHA,8.0,1000000,2
4,5d2737547bf280155f21609a,tech,Amsterdam,NLD,2007,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Wakoopa,8.0,1000000,6
5,5d2737547bf280155f21609c,education,Berlin,DEU,2007,"{'type': 'Point', 'coordinates': [13.38861, 52...",52.489700,13.388610,babbel,205.0,10000000,6
6,5d2737547bf280155f21609d,tech,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.0934254, ...",51.514157,-0.093425,Zemanta,30.0,7350000,17
7,5d2737547bf280155f21609e,tech,Madrid,ESP,2007,"{'type': 'Point', 'coordinates': [-3.6952896, ...",40.430940,-3.695290,eRepublik,30.0,2750000,3
8,5d2737547bf280155f2160a4,tech,Vancouver,CAN,2007,"{'type': 'Point', 'coordinates': [-123.1078045...",49.282108,-123.107805,Strutta,12.0,1600000,6
9,5d2737547bf280155f2160a5,news,Paris,FRA,2008,"{'type': 'Point', 'coordinates': [2.350987, 48...",48.856667,2.350987,fotopedia,17.0,12700000,11


In [268]:
data_companies = data_companies.sort_values(by=['number of offices near'], ascending=False)

In [269]:
data_companies['money by offices'] = data_companies['total_money_raised']/data_companies['number of offices near']

In [270]:
data_companies = data_companies.sort_values(by=['money by offices'], ascending=False) #columna dinero por compañia cercana. Creo nueva columna

In [271]:
display(data_companies)

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised,number of offices near,money by offices
50,5d2737547bf280155f2160e5,tech,Dublin,IRL,2007,"{'type': 'Point', 'coordinates': [-6.2689465, ...",53.343830,-6.268947,Green Apple Media,50.0,132000000,1,1.320000e+08
68,5d2737547bf280155f216109,social,Vancouver,CAN,2008,"{'type': 'Point', 'coordinates': [-123.0952381...",49.285173,-123.095238,HootSuite,370.0,190000000,6,3.166667e+07
70,5d2737547bf280155f216110,travel,Montreal,CAN,2007,"{'type': 'Point', 'coordinates': [-73.5550157,...",45.512433,-73.555016,Hopper,17.0,22000000,1,2.200000e+07
30,5d2737547bf280155f2160c6,travel,Zurich,CHE,2009,"{'type': 'Point', 'coordinates': [8.5082891, 4...",47.391290,8.508289,GetYourGuide,75.0,16000000,1,1.600000e+07
40,5d2737547bf280155f2160d4,tech,Toronto,CAN,2008,"{'type': 'Point', 'coordinates': [-79.567221, ...",43.644678,-79.567221,Wishabi,50.0,16000000,1,1.600000e+07
49,5d2737547bf280155f2160e4,tech,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.468617, 5...",51.518101,-0.468617,mydeco,35.0,12500000,1,1.250000e+07
66,5d2737547bf280155f216106,tech,Perth,AUS,2007,"{'type': 'Point', 'coordinates': [115.89084, -...",-31.978220,115.890840,HealthEngine,12.0,10400000,1,1.040000e+07
57,5d2737547bf280155f2160f1,tech,Tartu,EST,2007,"{'type': 'Point', 'coordinates': [26.7212208, ...",58.375411,26.721221,Fortumo,45.0,10000000,1,1.000000e+07
44,5d2737547bf280155f2160d9,tech,Copenhagen V,DNK,2007,"{'type': 'Point', 'coordinates': [12.5605372, ...",55.673962,12.560537,Trustpilot,100.0,18400000,2,9.200000e+06
62,5d2737547bf280155f2160f9,tech,Kitchener,CAN,2008,"{'type': 'Point', 'coordinates': [-80.487328, ...",43.449695,-80.487328,IGLOO Software,50.0,9000000,1,9.000000e+06


In [272]:
def sumEmployees(list_offices):
    list_employee = []
    res=0
    for company in list_offices:
        lst=[]
        for i in company:
            lst.append(i['number_of_employees'])
        res+=i['number_of_employees']
        list_employee.append(lst)
    result = [sum(b) for b in list_employee]
    return result

#sum(list_employee[0])

In [273]:
print(sumEmployees(list_offices))

[448.0, 119.0, 140.0, 14.0, 72.0, 435.0, 405.0, 70.0, 471.0, 272.0, 405.0, 119.0, 12.0, 88.0, 382.0, 140.0, 10.0, 45.0, 29.0, 72.0, 471.0, 54.0, 72.0, 386.0, 452.0, 112.0, 50.0, 54.0, 440.0, 8.0, 75.0, 72.0, 119.0, 87.0, 8.0, 119.0, 87.0, 72.0, 30.0, 440.0, 50.0, 394.0, 394.0, 162.0, 108.0, 5.0, 272.0, 342.0, 252.0, 35.0, 50.0, 70.0, 6.0, 13.0, 452.0, 12.0, 3.0, 45.0, 6.0, 136.0, 20.0, 390.0, 50.0, 17.0, 452.0, 272.0, 12.0, 40.0, 471.0, 212.0, 17.0, 214.0, 15.0, 272.0, 435.0, 72.0, 386.0, 190.0, 452.0, 498.0, 25.0, 87.0, 15.0, 30.0, 452.0, 471.0]


In [274]:
data_companies['employee by offices'] = sumEmployees(list_offices)/data_companies['number of offices near']

In [275]:
display(data_companies)

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised,number of offices near,money by offices,employee by offices
50,5d2737547bf280155f2160e5,tech,Dublin,IRL,2007,"{'type': 'Point', 'coordinates': [-6.2689465, ...",53.343830,-6.268947,Green Apple Media,50.0,132000000,1,1.320000e+08,448.000000
68,5d2737547bf280155f216109,social,Vancouver,CAN,2008,"{'type': 'Point', 'coordinates': [-123.0952381...",49.285173,-123.095238,HootSuite,370.0,190000000,6,3.166667e+07,19.833333
70,5d2737547bf280155f216110,travel,Montreal,CAN,2007,"{'type': 'Point', 'coordinates': [-73.5550157,...",45.512433,-73.555016,Hopper,17.0,22000000,1,2.200000e+07,140.000000
30,5d2737547bf280155f2160c6,travel,Zurich,CHE,2009,"{'type': 'Point', 'coordinates': [8.5082891, 4...",47.391290,8.508289,GetYourGuide,75.0,16000000,1,1.600000e+07,14.000000
40,5d2737547bf280155f2160d4,tech,Toronto,CAN,2008,"{'type': 'Point', 'coordinates': [-79.567221, ...",43.644678,-79.567221,Wishabi,50.0,16000000,1,1.600000e+07,72.000000
49,5d2737547bf280155f2160e4,tech,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.468617, 5...",51.518101,-0.468617,mydeco,35.0,12500000,1,1.250000e+07,435.000000
66,5d2737547bf280155f216106,tech,Perth,AUS,2007,"{'type': 'Point', 'coordinates': [115.89084, -...",-31.978220,115.890840,HealthEngine,12.0,10400000,1,1.040000e+07,405.000000
57,5d2737547bf280155f2160f1,tech,Tartu,EST,2007,"{'type': 'Point', 'coordinates': [26.7212208, ...",58.375411,26.721221,Fortumo,45.0,10000000,1,1.000000e+07,70.000000
44,5d2737547bf280155f2160d9,tech,Copenhagen V,DNK,2007,"{'type': 'Point', 'coordinates': [12.5605372, ...",55.673962,12.560537,Trustpilot,100.0,18400000,2,9.200000e+06,235.500000
62,5d2737547bf280155f2160f9,tech,Kitchener,CAN,2008,"{'type': 'Point', 'coordinates': [-80.487328, ...",43.449695,-80.487328,IGLOO Software,50.0,9000000,1,9.000000e+06,272.000000


In [285]:
data_companies = data_companies[data_companies['number of offices near'] > 1]

In [286]:
display(data_companies)

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised,number of offices near,money by offices,employee by offices,ranking
68,5d2737547bf280155f216109,social,Vancouver,CAN,2008,"{'type': 'Point', 'coordinates': [-123.0952381...",49.285173,-123.095238,HootSuite,370.0,190000000,6,3.166667e+07,19.833333,16.333333
44,5d2737547bf280155f2160d9,tech,Copenhagen V,DNK,2007,"{'type': 'Point', 'coordinates': [12.5605372, ...",55.673962,12.560537,Trustpilot,100.0,18400000,2,9.200000e+06,235.500000,97.000000
77,5d2737547bf280155f21611d,tech,Sao Paulo,BRA,2009,"{'type': 'Point', 'coordinates': [-46.6388182,...",-23.548943,-46.638818,Brandsclub,140.0,17000000,2,8.500000e+06,59.500000,26.600000
25,5d2737547bf280155f2160bc,tech,Budapest,HUN,2009,"{'type': 'Point', 'coordinates': [19.059078, 4...",47.490346,19.059078,Prezi,110.0,14300000,2,7.150000e+06,44.000000,20.400000
39,5d2737547bf280155f2160d3,social,Berlin,DEU,2008,"{'type': 'Point', 'coordinates': [13.3842605, ...",52.530852,13.384260,ResearchGate,74.0,35000000,7,5.000000e+06,1.428571,10.371429
27,5d2737547bf280155f2160bf,tech,Munich,DEU,2007,"{'type': 'Point', 'coordinates': [11.5893956, ...",48.165421,11.589396,AdScale,40.0,6150000,2,3.075000e+06,27.000000,13.600000
13,5d2737547bf280155f2160aa,social,Buenos Aires,ARG,2007,"{'type': 'Point', 'coordinates': [-58.417309, ...",-34.611781,-58.417309,Sonico,84.0,6000000,2,3.000000e+06,36.000000,17.200000
85,5d2737547bf280155f216128,tech,Vancouver,CAN,2009,"{'type': 'Point', 'coordinates': [-123.109217,...",49.282455,-123.109217,Tiny Speck,45.0,17200000,6,2.866667e+06,75.333333,38.533333
15,5d2737547bf280155f2160ac,enterprise,Toronto,CAN,2008,"{'type': 'Point', 'coordinates': [-79.406775, ...",43.643080,-79.406775,ScribbleLive,45.0,12000000,5,2.400000e+06,22.400000,15.960000
75,5d2737547bf280155f21611a,tech,Amsterdam,NLD,2009,"{'type': 'Point', 'coordinates': [4.8928722, 5...",52.365758,4.892872,Layar,32.0,13400000,6,2.233333e+06,8.333333,11.733333


In [ ]:
#df = pd.read_csv('../visualizing-real-world-data-project/list_offices.csv')
#data_companies.to_json('./results_final.json', orient='records')

In [279]:
def rank(data,colum):
    if data[colum]=="total_money_raised":
        data[colum] = data[colum]*0.8
    elif data[colum]== "number of offices near":
        data[colum] = data[colum]*0.6
    elif data[colum]=="number of employees":
        data[colum] = data[colum]*0.4
    return data[colum].sum()
    
    

In [287]:
data_companies['ranking'] = data_companies['number of offices near']*0.8 + data_companies['number of offices near']*0.6 + data_companies['employee by offices']*0.4

In [288]:
data_companies = data_companies.sort_values(['ranking'], ascending=False)

In [289]:
display(data_companies)

,_id,category_code,city,country,founded_year,geoDescription,latitude,longitude,name,number_of_employees,total_money_raised,number of offices near,money by offices,employee by offices,ranking
44,5d2737547bf280155f2160d9,tech,Copenhagen V,DNK,2007,"{'type': 'Point', 'coordinates': [12.5605372, ...",55.673962,12.560537,Trustpilot,100.0,18400000,2,9.200000e+06,235.500000,97.000000
3,5d2737547bf280155f216099,tech,Helsinki,FIN,2007,"{'type': 'Point', 'coordinates': [24.9410012, ...",60.168149,24.941001,XIHA,8.0,1000000,2,5.000000e+05,195.000000,80.800000
83,5d2737547bf280155f216126,tech,Brussels,BEL,2009,"{'type': 'Point', 'coordinates': [4.3464721, 5...",50.839970,4.346472,Getyoo,10.0,1130000,3,3.766667e+05,157.000000,67.000000
19,5d2737547bf280155f2160b6,tech,Stockholm,SWE,2007,"{'type': 'Point', 'coordinates': [18.064488, 5...",59.332788,18.064488,Bambuser,12.0,2500000,3,8.333333e+05,114.000000,49.800000
32,5d2737547bf280155f2160c9,tech,Ramat Gan,ISR,2008,"{'type': 'Point', 'coordinates': [34.8003705, ...",32.037363,34.800370,Tweegee,22.0,3000000,5,6.000000e+05,90.400000,43.160000
85,5d2737547bf280155f216128,tech,Vancouver,CAN,2009,"{'type': 'Point', 'coordinates': [-123.109217,...",49.282455,-123.109217,Tiny Speck,45.0,17200000,6,2.866667e+06,75.333333,38.533333
5,5d2737547bf280155f21609c,education,Berlin,DEU,2007,"{'type': 'Point', 'coordinates': [13.38861, 52...",52.489700,13.388610,babbel,205.0,10000000,6,1.666667e+06,73.333333,37.733333
41,5d2737547bf280155f2160d5,social,London,GBR,2007,"{'type': 'Point', 'coordinates': [-0.0925694, ...",51.502767,-0.092569,SocialGO,12.0,3000000,17,1.764706e+05,29.294118,35.517647
64,5d2737547bf280155f2160fd,social,London,GBR,2008,"{'type': 'Point', 'coordinates': [-0.1046546, ...",51.522427,-0.104655,Mendeley,27.0,2130000,18,1.183333e+05,25.111111,35.244444
78,5d2737547bf280155f21611e,music,London,GBR,2008,"{'type': 'Point', 'coordinates': [-0.0856982, ...",51.525377,-0.085698,Musicmetric,15.0,3400000,18,1.888889e+05,25.111111,35.244444


In [ ]:
z = folium.Map(location=[51.510880, -0.141897], width=750, height=500, zoom_start=2)
for index, row in data_companies.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=3,
                        popup="Name company: {}. Money: {}".format(row['name'], row['total_money_raised']),
                        fill_color="#F35C50", # divvy color
                       ).add_to(z)
z

In [ ]:
def mapLocations(list_offices):
    global coord
    list_coordinates = []
    for e in range(len(list_offices)):
        list_2 = list_offices[e]['geoDescription']['coordinates']
        list_coordinates.append(list_2)
    map_offices = folium.Map(location=[coord[1], coord[0]], zoom_start=2, )
    for coord in list_coordinates:
        folium.CircleMarker([coord[1], coord[0]], radius=5, parse_html=False).add_to(map_offices, )
    map_offices.save( 'map.html')
    return map_offices
#map_offices
mapLocations(list_offices)

In [ ]:
    #for coord in lista:
        #print(coord)
        #mapit = folium.Map( location=[ coord[0], coord[1]])

In [ ]:
def mapLocations(coordinateslist):
    list_coordinates = []
    for e in range(len(list_offices)):
        list_2 = list_offices[e]['geoDescription']['coordinates']
        list_coordinates.append(list_2)

    map_offices = folium.Map(location=[coord[1], coord[0]], zoom_start=10)

    for coord in list_coordinates:
        folium.CircleMarker([coord[1], coord[0]], radius=5, parse_html=False).add_to(map_offices)
    map_offices.save( 'map.html')

map_offices

In [ ]:
#companies = db.data_companies_clean.find(
    #{
    #"country":{
            #"$eq":"ESP"
        #}
    #}
    #{"founded_year":{"$gte": 2007 }}

#)
#data_companies = pd.DataFrame(companies)

#data_companies.head()

In [ ]:
data_companies['city'].value_counts().head()

In [ ]:
data_companies.groupby('city')["total_money_raised"].sum()

In [ ]:
data = data_companies.groupby('city', as_index=False)["total_money_raised"].mean()
data.head()

In [ ]:
data_companies.groupby('city').sum()["total_money_raised"]/len(data_companies['city'])

In [ ]:
df.groupby('StationID', as_index=False)['BiasTemp'].mean()